In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

## Create Data Frame

In [2]:
breath_df = pd.read_pickle('breath_pickle')

In [3]:
under2 = breath_df[abs(breath_df['Csv_breath_holds']-breath_df['Data_breath_holds'])<=2]

In [4]:
under2.head()

,Trace,Csv_breath_holds,Data_breath_holds,Full_trace,breathhold_idx,bh_start_end
0,"[0.0003, 0.0006, 0.001, 0.0014, 0.0019, 0.0023...",31.29,31.69,"[0.0003, 0.0006, 0.001, 0.0014, 0.0019, 0.0023...","[2661, 2662, 2663, 2664, 2665, 2666, 2667, 266...","(2661, 5829)"
1,"[0.0006, 0.0009, 0.0013, 0.0017, 0.0022, 0.002...",30.61,31.40,"[0.0006, 0.0009, 0.0013, 0.0017, 0.0022, 0.002...","[2386, 2387, 2388, 2389, 2390, 2391, 2392, 239...","(2386, 5525)"
3,"[0.0013, 0.0027, 0.0034, 0.0041, 0.005, 0.006,...",30.85,31.84,"[0.0013, 0.0027, 0.0034, 0.0041, 0.005, 0.006,...","[2573, 2574, 2575, 2576, 2577, 2578, 2579, 258...","(2573, 5756)"
4,"[0.0012, 0.0015, 0.0019, 0.0023, 0.0029, 0.003...",32.12,32.53,"[0.0012, 0.0015, 0.0019, 0.0023, 0.0029, 0.003...","[2520, 2521, 2522, 2523, 2524, 2525, 2526, 252...","(2520, 5772)"
5,"[0.0008, 0.0011, 0.0014, 0.0018, 0.0023, 0.002...",30.88,31.02,"[0.0008, 0.0011, 0.0014, 0.0018, 0.0023, 0.002...","[2683, 2684, 2685, 2686, 2687, 2688, 2689, 269...","(2683, 5784)"


In [15]:
under2 = under2[0:100]

In [6]:
def split_trace(sequence, n_steps, n_output):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-n_output:
            break

        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+n_output]

        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [7]:
num_outputs = 10

In [12]:
traces = []
next_pt = []
for row in under2['Trace']:
    Xs, ys = split_trace(row, 100, num_outputs)
    traces.append(Xs)
    next_pt.append(ys)
    

In [13]:
traces2 = [np.array(x, dtype='float32') for sublist in traces for x in sublist]
next_pt2 = [x for sublist in next_pt for x in sublist]

## Scale Data

In [ ]:
scaler = StandardScaler() # creates the scaler
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

## Initialize Data for Model

In [13]:
#dataset
from torch.utils.data import Dataset

In [14]:
class timeseries(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len

In [15]:
dataset = timeseries(x_train,y_train)
valid_dataset = timeseries(x_val,y_val)
test_dataset = timeseries(x_test,y_test)

In [16]:
train_loader = DataLoader(dataset,shuffle=False,batch_size=200)
val_loader = DataLoader(valid_dataset,shuffle=False,batch_size=200)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=200)

## Yannet's Suggestion
Add prediction to 'x' and make new prediciton.

In [17]:
#neural network
from torch import nn

class LSTM_Model(nn.Module):
    def __init__(self):
        super(LSTM_Model,self).__init__()
        self.lstm = nn.LSTM(input_size=1,hidden_size=5,num_layers=1,batch_first=True)
        self.linear = nn.Linear(in_features=5,out_features=1)
        
    def forward(self,x):
        output,_status = self.lstm(x)
        output = output[:,-1,:]
        output = self.linear(torch.relu(output))
        return output

In [18]:
model = torch.load('lstm_single_output_1000_epochs')

In [19]:
model

LSTM_Model(
  (lstm): LSTM(1, 5, batch_first=True)
  (linear): Linear(in_features=5, out_features=1, bias=True)
)

In [20]:
def make_pred(x_test, y_test, n_preds):
    
    test_set = timeseries(x_test,y_test)
    x = test_set[:][0]
   
    preds = []
    
    for i in range(10, 10+n_preds):
        test_pred = model(x.view(-1,i,1)).view(-1)
        preds.append(test_pred)
        x = torch.cat((x, test_pred.view(-1,1)), 1)
    
    return preds

In [21]:
preds = make_pred(x_test, y_test, 20)

In [25]:
y_test

array([-0.822081,  0.809876,  0.143455, ..., -0.472583, -0.144097,
        0.874191])

In [24]:
preds

[tensor([-0.8239,  0.8103,  0.1427,  ..., -0.4734, -0.1450,  0.8758],
        grad_fn=<ViewBackward0>),
 tensor([0.2459, 0.6077, 0.5805,  ..., 0.3583, 0.4773, 0.5938],
        grad_fn=<ViewBackward0>),
 tensor([0.4456, 0.5188, 0.7006,  ..., 0.5344, 0.6286, 0.4626],
        grad_fn=<ViewBackward0>),
 tensor([0.5635, 0.6180, 0.6355,  ..., 0.5929, 0.6117, 0.5988],
        grad_fn=<ViewBackward0>),
 tensor([0.6219, 0.7428, 0.5829,  ..., 0.6146, 0.5911, 0.7626],
        grad_fn=<ViewBackward0>),
 tensor([0.6612, 0.7239, 0.6287,  ..., 0.6570, 0.6423, 0.7329],
        grad_fn=<ViewBackward0>),
 tensor([0.6821, 0.6211, 0.7080,  ..., 0.6946, 0.7079, 0.5988],
        grad_fn=<ViewBackward0>),
 tensor([0.6801, 0.5754, 0.7183,  ..., 0.6936, 0.7112, 0.5491],
        grad_fn=<ViewBackward0>),
 tensor([0.6658, 0.6346, 0.6631,  ..., 0.6647, 0.6628, 0.6359],
        grad_fn=<ViewBackward0>),
 tensor([0.6568, 0.7070, 0.6193,  ..., 0.6448, 0.6277, 0.7281],
        grad_fn=<ViewBackward0>),
 tensor([0.659